In [ ]:
import faiss
import numpy as np
import pickle


In [ ]:
    
# Load data index
save_path = '/home/kylecshan/saved/06_01_2019/classification/densenet169/densenet'


# Get descriptor matrices
index = faiss.read_index(save_path + '.index')
test = faiss.read_index(save_path + 'test.index')

nb = index.ntotal
nq = test.ntotal
index_matrix = np.vstack([index.reconstruct(i) for i in range(nb)])
test_matrix = np.vstack([test.reconstruct(i) for i in range(nq)])


In [ ]:
print (index_matrix.shape)
print (test_matrix.shape)

In [ ]:
data_path = save_path + '.txt'

with open(data_path, "rb") as fp: 
    data_index, label_index = pickle.load(fp)

In [ ]:
print (len(data_index))
print (len(label_index))

In [ ]:
import sys
sys.path.insert(0,'/home/kylecshan/Kaushik/retrieval/')
from dataloader import load_data

dataloaders_dict = load_data(batch_size = 256)
label_test = []
img_id = []

flag = 0
for i, (_, labels, image_id) in enumerate(dataloaders_dict['test']):
    label_test.extend(labels.tolist())
    img_id.extend(list(image_id))
    

In [ ]:
print (len(label_test))
print (len(img_id))
print (img_id[0])

In [ ]:
# For each test image, find top K images in the index matrix 
top = 5
iterations = 3
write_path = '/home/kylecshan/saved/06_01_2019/classification/densenet169/QE/5_3'

sub = write_path + 'idx.csv'
class_file = write_path + 'label.csv'

with open(sub, 'ab') as f, open(class_file, 'ab') as g:
    for i in range(nq):
        query_new = test_matrix[i][np.newaxis, :]
        for it in range(iterations):
            D, I = index.search(query_new, top)
            query_new = np.mean(np.vstack((index_matrix[I].squeeze(), query_new)), axis = 0)[np.newaxis, :]
            assert query_new.shape[1] == 1664
            
        D, I = index.search(query_new, 100)
        np.savetxt(f, [['id', 'images']], delimiter = ',',fmt="%s")
        
        print (i)
        t_label = label_test[i]
        np.savetxt(g, [t_label], newline = ',', fmt="%s")
        indices = I.squeeze()
        neighbors_labels = [[label_index[ind] for ind in indices]]
        np.savetxt(g, neighbors_labels, delimiter = ',', fmt="%s")

        np.savetxt(f, [img_id[i]], newline = ',', fmt="%s")
        neighbors = [[data_index[ind] for ind in indices]]
        np.savetxt(f, neighbors, delimiter = ' ', fmt="%s")






In [ ]:
# Use different distance metrics

from scipy.spatial import distance

write_path = '/home/kylecshan/saved/06_04_2019/delfTL_dist/cosine/delfTL'

sub = write_path + 'idx.csv'
class_file = write_path + 'label.csv'

k = 100
with open(sub, 'ab') as f, open(class_file, 'ab') as g:
    for i in range(nq):
        outputs = test_matrix[i][np.newaxis, :]
        D = distance.cdist(outputs, index_matrix, 'cosine')
        I = np.argsort(D, axis = 1)[:,:k]
        np.savetxt(f, [['id', 'images']], delimiter = ',',fmt="%s")
        
        print (i)
        t_label = label_test[i]
        np.savetxt(g, [t_label], newline = ',', fmt="%s")
        indices = I.squeeze()
        neighbors_labels = [[label_index[ind] for ind in indices]]
        np.savetxt(g, neighbors_labels, delimiter = ',', fmt="%s")

        np.savetxt(f, [img_id[i]], newline = ',', fmt="%s")
        neighbors = [[data_index[ind] for ind in indices]]
        np.savetxt(f, neighbors, delimiter = ' ', fmt="%s")




